In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain.prompts import ChatPromptTemplate
import re
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI



In [ ]:
# model = "llama3.1"
# # model = "mistral-nemo"

In [ ]:
# # Initialize LLM
# llm = Ollama(model=model)
# embeddings = OllamaEmbeddings(model="llama3.1")


In [ ]:
model = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-3.5-turbo")
# model = "llama, grok, mistral, etc"

In [ ]:
pdf_path_1 = r"C:\Users\FRD_Test_Policy\NIST.IR.8259.pdf"
pdf_path_2 = r"C:\Users\FRD_Test_Policy\PLAW-116publ207.pdf"

In [ ]:
frd_1_loader = PyPDFLoader(pdf_path_1)
pages_1 = frd_1_loader.load()

frd_2_loader = PyPDFLoader(pdf_path_2)
pages_2 = frd_2_loader.load()

In [ ]:
# Split the document into manageable chunks
text_splitter_1 = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
text_1 = text_splitter_1.split_documents(pages_1)

text_splitter_2 = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
text_2 = text_splitter_2.split_documents(pages_2)

In [ ]:
frd_1 = [doc.page_content for doc in text_1]
frd_2 = [doc.page_content for doc in text_2]

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [ ]:
vectorstore_1 = DocArrayInMemorySearch.from_texts(frd_1, embedding=embeddings)
vectorstore_2 = DocArrayInMemorySearch.from_texts(frd_2, embedding=embeddings)

In [ ]:
parser = StrOutputParser()
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model | parser 

In [ ]:
# query embedding
query_embedding = embeddings.embed_query(query)

# similarity search in the vector store
results = vectorstore_1.similarity_search(query, k=8)  

In [ ]:
# chain with context and query
response = chain.invoke({
    "context": context,
    "question": query
})

In [ ]:
#

In [ ]:
# call vecstore 2 
vectorstore_2 = DocArrayInMemorySearch.from_texts(frd_2, embedding=embeddings)

# Template for Entity and Relationship Extraction
template = """
Based on the document titled "Internet of Things Cybersecurity Improvement Act of 2020" which establishes standards for securing IoT devices used by federal agencies. 

Your task is to extract:
1. Key Entities or Classes: Identify the main categories or entities (e.g., "IoT Devices," "Manufacturers," "Federal Agencies," "Cybersecurity Standards," etc.) that the document focuses on in each section.
2. Relationships: identify in triples how these entities are connected or interact with one another as per the document. For example:
   - "Manufacturers "must adhere to" Cybersecurity Standards."
   

Output Format: 
1. [Entity/Class Name]: 
2. Relationships: in triples of [subject, predicate, object] 
      [Entity A → Relationship → Entity B]
      [Entity C → Relationship → Entity D]

Context: {context}

Please go through the whole document and extensively identify the key entities or classes and their respective relationships based on the provided context.
"""

# prompt template for the task
frd_2_entity_relationship_prompt = ChatPromptTemplate.from_template(template)

# Set up the chain
frd_2_entity_relationship_chain = frd_2_entity_relationship_prompt | model | parser

# Query for extracting key entities and relationships
query = "Identify the key entities or classes and their relationships in the IoT Cybersecurity Improvement Act of 2020."

# similarity search to retrieve relevant context from the vectorstore
query_embedding = embeddings.embed_query(query)
results = vectorstore_2.similarity_search_by_vector(query_embedding, k=16)  # Top 6 relevant sections
context = " ".join([result.page_content for result in results])  # Combine retrieved content
response = frd_2_entity_relationship_chain.invoke({
    "context": context,
})

print("FRD_2 Key Entities and Relationships:")
print(response)


In [ ]:
# call vecstore 1 
vectorstore_1 = DocArrayInMemorySearch.from_texts(frd_1, embedding=embeddings)

# Template for Entity Extraction
template = """

The document titled "NIST 8259" provides guidance for IoT device manufacturers to enhance cybersecurity capabilities. 

Your task is to extract:
1. Key Entities or Classes: Identify the main categories or entities (e.g., "IoT Devices," "Manufacturers," "Federal Agencies," "Cybersecurity Standards," etc.) that the document focuses on in each section.
2. Relationships: identify in triples how these entities are connected or interact with one another as per the document. For example:
   - "Manufacturers "must adhere to" Cybersecurity Standards."
   

Output Format: 
1. Entity/Class Name: 
2. Relationships: in triples of [subject, predicate, object] 
      [Entity A → Relationship → Entity B]
      [Entity C → Relationship → Entity D]

Context: {context}

Please go through the whole document and extensively identify the key entities or classes and their respective relationships based on the provided context.
"""

# prompt template for the task
frd_1_entity_relationship_prompt = ChatPromptTemplate.from_template(template)

# Set up the chain
frd_1_entity_relationship_chain = frd_1_entity_relationship_prompt | model | parser

# Query for extracting key entities and relationships
query = "Identify the key entities or classes and their relationships in NIST 8259."

# Perform similarity search to retrieve relevant context from the vectorstore
query_embedding = embeddings.embed_query(query)
results = vectorstore_1.similarity_search_by_vector(query_embedding, k=16)  # Top 6 relevant sections
context = " ".join([result.page_content for result in results])  # Combine retrieved content
response = frd_1_entity_relationship_chain.invoke({
    "context": context,
})

print("FRD_1 Key Entities and Relationships: ")
print(response)

In [ ]:
# 

In [ ]:
# ontology
from owlready2 import *

# Create a new ontology
onto = get_ontology("http://example.org/onto.owl")

In [ ]:
with onto:

# Define the core class (Stakeholder)
    class Stakeholder(Thing):
        namespace = onto

    # Subclasses of Stakeholder with their respective subclasses if any 
    class Manufacturer(Stakeholder):
        namespace = onto        
    
    class RegulatoryBody(Stakeholder):
        namespace = onto
        
    class Standards(RegulatoryBody):   
        namespace = onto           
        
    # Define the Process class as independent
    class Process(Thing):
        namespace = onto
    class CoreBaseline(Process):
        namespace = onto
    class ComplianceCheck(Process): 
        namespace = onto
    class cybersecurityRiskManagementApproaches(Process):
        namespace = onto
    class ConfigurationManagement(Process):
        namespace = onto
    class Updates(Process):
        namespace = onto
    class MinimumSecurityStandards(Process):
        namespace = onto
        
    # Define the Means class as independent
    class Means(Thing):
        namespace = onto  

    #subclasses of Means 
    class IoTDevice(Means):
        namespace = onto
    class IoTPlatform(Means):
        namespace = onto
    class SecureSoftwareArchitecture(Means):
        namespace = onto
    class CybersecurityCapability(Means):
        namespace = onto
    class NonCybersecurityCapability(Means):
        namespace = onto
    class networkInterface(Means):  
        namespace = onto
    class Transducer(Means):
        namespace = onto
    class PatchAvailability (Means):
        namespace = onto
    class SecureDevelopment(Means):
        namespace = onto
    class ConventionalITDevice(Means):
        namespace = onto
    class NonCompliantIoTDevices(Means):
        namespace = onto
        
    # Define relationships   
    class include(ObjectProperty):
        domain = [Process]
        range = [Means]
    class contain(ObjectProperty):
        domain = [Means]
        range = [Process]
    class include(ObjectProperty):
        domain = [Process]
        range = [Process]

    class encompasses(ObjectProperty):
        domain = [CoreBaseline]
        range = [CybersecurityCapability]

    class doesNotInclude(ObjectProperty):
        domain = [CoreBaseline]
        range = [NonCybersecurityCapability]

    class derivedFrom(ObjectProperty):
        domain = [CoreBaseline]
        range = [cybersecurityRiskManagementApproaches]  

    class isResponsibleFor(ObjectProperty):
        domain = [RegulatoryBody]
        range = [ComplianceCheck]

    class developsAndUpdates(ObjectProperty):
        domain = [RegulatoryBody]
        range = [Standards]

    class definesAndUpdates(ObjectProperty):
        domain = [Standards]
        range = [CoreBaseline]    

    class performs(ObjectProperty):
        domain = [Manufacturer]
        range = [Process]
    class shouldImplement(ObjectProperty):
        domain = [Manufacturer]
        range = [Process]  
    class shouldEstablish(ObjectProperty):
        domain = [Manufacturer]
        range = [Process]
    class identify(ObjectProperty):
        domain = [Manufacturer]
        range = [Customer]
    class define(ObjectProperty):
        domain = [Manufacturer]
        range = [Process]
    class determines(ObjectProperty):
        domain = [Manufacturer]
        range = [Process]
    class researches(ObjectProperty):
        domain = [Manufacturer]
        range = [Customer]
    class understands(ObjectProperty):
        domain = [Manufacturer]
        range = [Process]

    class provides(ObjectProperty):
        domain = [Manufacturer]
        range = [Process]
    class plansFor(ObjectProperty):
        domain = [Manufacturer]
        range = [Process]
    class provisions(ObjectProperty):
        domain = [Manufacturer]
        range = [Process]
    class considers(ObjectProperty):
        domain = [Manufacturer]
        range = [Process]
    class developsStrategies(ObjectProperty):
        domain = [Manufacturer]
        range = [Process]
    class decides(ObjectProperty):
        domain = [Manufacturer]
        range = [Process]
    class determinesContent(ObjectProperty):
        domain = [Manufacturer]
        range = [Process]

    class appliesTo(ObjectProperty):
        domain = [Process]
        range = [Means]    
    class performedBy(ObjectProperty):
        domain = [Process]
        range = [Means]     

    # ObjectProperty Relationships
    class canHelpManufacturersIdentify(ObjectProperty):
        domain = [CoreBaseline]
        range = [CybersecurityCapability]

    class willNeed(ObjectProperty):
        domain = [IoTDevice]
        range = [CybersecurityCapability]

    class identifiedCybersecurityCapabilities(DataProperty):
        domain = [CoreBaseline]
        range = [str]  

    class shapedByCustomerRequirements(DataProperty):
        domain = [CybersecurityCapability]
        range = [str]  

    class requiredCybersecurityCapabilities(DataProperty):
        domain = [IoTDevice]
        range = [str]  

    class actionRequiredStatus(DataProperty):
        domain = [Updates]  
        range = [bool]  

    # PLaw ACT 2020 integration		
    # Manufacturers and IoT Devices

    class develop(ObjectProperty):
        domain = [Manufacturer]
        range = [IoTDevice]

    class adhereTo(ObjectProperty):
        domain = [Manufacturer]
        range = [Standards]

    class mustEnsure(ObjectProperty):
        domain = [Manufacturer]
        range = [PatchAvailability]#PatchAvailability 

    class mustDesignFor(ObjectProperty):
        domain = [Manufacturer]
        range = [SecureDevelopment]#SecureDevelopment

    class mustCollaborateWith(ObjectProperty):
        domain = [Manufacturer]
        range = [Organizations] #FederalAgency

    # IoT Devices
    class areDefinedBy(ObjectProperty):
        domain = [IoTDevice]
        range = [Standards] 

    class mustMeet(ObjectProperty):
        domain = [IoTDevice]
        range = [MinimumSecurityStandards]

    class mustComplyWith(ObjectProperty):
        domain = [IoTDevice]
        range = [Standards]

    class have(ObjectProperty):
        domain = [IoTDevice]
        range = [networkInterface]
        
    class have(ObjectProperty):
        domain = [IoTDevice]
        range = [Transducer]

    class differFrom(ObjectProperty):
        domain = [IoTDevice]
        range = [ConventionalITDevice]

    # Regulatory Bodies class develop(ObjectProperty):
    class oversees(ObjectProperty):
        domain = [Organizations]
        range = [Process]

    class develops(ObjectProperty):
        domain = [Standards] #Director of the Institute
        range = [Standards] # Standards and guidelines

    class reviews(ObjectProperty):
        domain = [RegulatoryBody]
        range = [Process]

    class establishes(ObjectProperty):
        domain = [RegulatoryBody]
        range = [Standards]

    class considers(ObjectProperty):
        domain = [Standards]
        range = [Standards]

    class consultsWith(ObjectProperty):
        domain = [Organizations] # 
        range = [Organizations]

    class consultsWith(ObjectProperty):
        domain = [RegulatoryBody] 
        range = [Organizations] # CybersecurityResearchers, IndustryExperts - customers

    class publishes(ObjectProperty):
        domain = [RegulatoryBody]
        range = [Standards]

    class provideAssistanceTo(ObjectProperty):
        domain = [Organizations] # SecretaryOfHomelandSecurity - gov agency under organization
        range = [RegulatoryBody]

    # Customers and Federal Agencies
    class use(ObjectProperty):
        domain = [Organizations] #FederalAgency
        range = [IoTDevice]

    class mustDisclose(ObjectProperty):
        domain = [Organizations]
        range = [Process]#CybersecurityVulnerability 

    class cannotProcure(ObjectProperty):
        domain = [Organizations] #FederalAgency
        range = [Means]#NonCompliantIoTDevices 

    class mustCoordinate(ObjectProperty):
        domain = [Organizations] #FederalAgency
        range = [Process]#DisclosureProcess

    class mustFollow(ObjectProperty):
        domain = [Organizations] #FederalAgency
        range = [Standards] #OMBPolicy
        
    class mustComplyWith (ObjectProperty):
        domain = [Organizations] #FederalAgency
        range = [Standards] #NIST Guidelines

    class mustImplement(ObjectProperty):
        domain = [Organizations] #FederalAgency
        range = [Process] #ConfigurationManagement

    class collaborateWith(ObjectProperty):
        domain = [Organizations] #FederalAgency
        range = [Customer] #Industry, Academia (customers)

    class ownOrControl(ObjectProperty):
        domain = [Organizations] #FederalAgency
        range = [IoTDevice]

    # Miscellaneous
    class grant(ObjectProperty):
        domain = [Standards] #Waiver
        range = [Organizations] #NationalSecurityResearch

    class submitsReportsTo(ObjectProperty):
        domain = [Organizations] #ComptrollerGeneral
        range = [Organizations] #Committees

    class address(ObjectProperty):
        domain = [Organizations] #Committees
        range = [IoTDevice] #IoTEfforts

    class assess(ObjectProperty):
        domain = [RegulatoryBody] #GAO
        range = [Process] #IoTSecurityVulnerabilities

    class submitReport(ObjectProperty):
        domain = [Standards] #GAOReport
        range = [Organizations] #CongressionalCommittees

    class isTaskedToDevelop(ObjectProperty):
        domain = [RegulatoryBody]
        range = [Standards] #Guideline

    class mustAlignWith(ObjectProperty):
        domain = [Standards]
        range = [Standards] # ISOStandard

    class mustReview(ObjectProperty):
        domain = [RegulatoryBody]
        range = [Process] #TimePeriod

    class mustConsider(ObjectProperty):
        domain = [RegulatoryBody]
        range = [Standards] # BestPractice

    class reviews(ObjectProperty):
        domain = [Organizations]  # DirectorOMB
        range = [Standards] #Policy

    class ensuresAlignmentWith(ObjectProperty): 
        domain = [Organizations]  # DirectorOMB
        range = [Standards] #IoTStandard

    class issues(ObjectProperty):
        domain = [Organizations] # DirectorOMB
        range = [Standards] # Policy, Principle

    class ensureConsistencyWith(ObjectProperty):
        domain = [Standards] # Policy, Principle
        range = [Standards] # standards and guidelines
        
    class establishes(ObjectProperty):
        domain = [Organizations] #OMB
        range = [Process]

    class mustReport(ObjectProperty):
        domain = [Customer] # Contractor
        range = [Process] # SecurityVulnerability

    class mustComplyWith(ObjectProperty):
        domain = [Customer] #Contractor
        range = [Standards] # CybersecurityStandard

    class mustAlignWith(ObjectProperty):
        domain = [Customer] #Contractor
        range = [Standards] #FederalRegulation

    class assesses(ObjectProperty):
        domain = [Organizations] # GAO gov agency under organization
        range = [Process] # IoTVulnerability

    class isSubmittedTo(ObjectProperty):
        domain = [Standards] # report under - standard
        range = [Organizations] # committee - gov agency under organization
        
    class receiveReportsFrom(ObjectProperty):	
        domain = [Organizations] # report under - standard
        range = [Organizations] # committee - gov agency under organization
        
    class mustAddress(ObjectProperty):
        domain = [Standards] #FederalAcquisitionRegulation 
        range = [Process] #SecurityVulnerability

    class canBeGrantedFor(ObjectProperty):
        domain = [Standards] #Waiver under standards 
        range = [Process]#NationalSecurity

    class canBeGrantedFor(ObjectProperty):
        domain = [Standards] #Waiver under standards 
        range = [Organizations]#Research - educational_institutions

    class canBeGrantedFor(ObjectProperty):
        domain = [Standards] #Waiver under standards 
        range = [Individuals]#cybersec Researchers

    class review(ObjectProperty):
        domain = [Organizations] # ChiefInformationOfficer - gov agency under organization
        range = [Process] #Compliance

    class passes(ObjectProperty):
        domain = [Organizations] # congress - gov agency under Organizations
        range = [Standards] #Act

    class establishes(ObjectProperty):
        domain = [Standards] #IoTCybersecurityImprovementAct
        range = [Process] #MinimumSecurityStandard ctbersecurity baseline 
        
    class doNotApplyTo(ObjectProperty):
        domain = [Organizations] #OMB Policies
        range = [Means] #National Security Systems 

    # Define Data Properties for Process descriptions 
    class data_property(DataProperty):
        namespace = onto
        domain = [Manufacturer]
        range = [str]

    class data_description(DataProperty):
        namespace = onto
        domain = [Manufacturer]
        range = [str]

    class object_property(DataProperty):
        namespace = onto
        domain = [Manufacturer]
        range = [str]

    class object_description(DataProperty):
        namespace = onto
        domain = [Manufacturer]
        range = [str]

    class policy_description(DataProperty):
        namespace = onto
        domain = [Manufacturer]
        range = [str]

    class status(DataProperty):
        namespace = onto
        domain = [Manufacturer]
        range = [str]
        
    class compliance_report(DataProperty):
        domain = [Manufacturer]
        range = [str]

    class Timestamp(DataProperty):
        namespace = onto
        domain = [Manufacturer]
        range = [str]

    # Define specific properties for concepts
    class expected_use_case(DataProperty):
        domain = [IoTDevice]
        range = [str]

    class vulnerabilities(DataProperty):
        domain = [IoTDevice]
        range = [str]

    class updates(DataProperty):
        domain = [IoTDevice]
        range = [str]

# Save ontology
onto.save(file="n8259a.owl", format="rdfxml")

In [ ]:

onto.load()

# Create instances for Stakeholders
manufacturer = Manufacturer("Manufacturer-1")

small_retail_businesses = Organizations("Small_retail_businesses")

hospitals = Organizations("Hospitals")
energy_companies = Organizations("Energy_companies")

national_security_research = Organizations("National_Security_Research")
educational_institutions = Organizations("Educational_Institutions")

government_agencies = Organizations("Government_Agencies")
secretary_HS = Organizations("Secretary_of_Homeland_Security")
compt_USA = Organizations("Comptroller_General_of_the_United_States")
dir_OMB = Organizations("Director_of_the_Office_of_Management_and_Budget")
GAO = Organizations("Government_accountability_office")

CIO = Organizations("chief_information_officer")
congress = Organizations("congress")
congressional_committees = Organizations("congressional_committees")

NIST = RegulatoryBody("National_Institute_of_Standards_and_Technology")
dir_NIST = RegulatoryBody("Director_of_the_National_Institute_of_Standards_and_Technology")

iot_cyber_act_2020 = Standards("Internet_of_Things_Cybersecurity_Improvement_Act_of_2020")
cybersecurity_standards = Standards("Cybersecurity_Standards")
ISO_standards_29147_and_30111 = Standards("ISO_standards_29147_and_30111")
IoT_security_standards_and_guidelines = Standards("IoT_security_standards_and_guidelines")
minimum_security_standards = Standards("Minimum_security_standards")

NIST_IR_8259 = Standards("NIST.IR.8259")
NIST_draft_report = Standards("NIST_draft_report")
NIST_guidelines = Standards("NIST_guidelines")
security_guidelines = Standards("Security_Guidelines")
waivers = Standards("Waivers")

IoT_cybersecurity_improvement_act = Standards("IoT_cybersecurity_improvement_act_of_2020")
act = Standards("Act")
Federal_acquisition_regulation = Standards("Federal_acquisition_regulation")
GAO_report = Standards("GAO_Report")
Federal_regulation = Standards("Federal_regulation")

best_practices = Standards("Best_practices")
report = Standards("Report")
security_vulnerabilities_reporting_guidelines = Standards("security_vulnerabilities_reporting_guidelines")

OMB_policies = Standards("OMB_policies") 

agency_security_policies = Standards("agency_security_policies") 
policies_and_principles = Standards("Policies_and_Principles")

# Create instances of Means

integrator = Means("Integrator")
firewalls = Means("firewalls")
IDSs = Means("IDS")

national_security_systems = Means("National_Security_Systems")

hardware_and_software_resources = SecureSoftwareArchitecture("secure_software_architecture")
secure_software_architecture = SecureSoftwareArchitecture("secure_software_architecture")

non_cybersecurity_capability = NonCybersecurityCapability("non_cybersecurity_device_capabilities_that_support_privacy")
cybersecurity_capability = CybersecurityCapability("Cybersecurity_Capability")

iot_platform = IoTPlatform("IoT_Platform")
iot_device = IoTDevice("IoT_Device")
iot_device_data = IoTDevice("IoT_Device_data")
IoT_efforts = IoTDevice("IoT_Efforts")

transducer = Transducer("sensor_or_actuator")

non_compliant_IoT_devices = NonCompliantIoTDevices("Non_compliant_IoT_devices")
conventional_IT_devices = ConventionalITDevice("Conventional_IT_devices")
secure_development_processes = SecureDevelopment("Secure_development_processes")
patch_availability = PatchAvailability("Patch_availability")

# Create instances of networkInterface

network_interface = networkInterface("Ethernet_LTE_UWB")
wifi = networkInterface("Wi-Fi")
zigbee = networkInterface("ZigBee")
bluetooth = networkInterface("Bluetooth")
  
# Create instances for Processes
configuration_management = ConfigurationManagement("configuration_management")
cybersecurity_configuration_settings = ConfigurationManagement("configuration_management")

compliance = ComplianceCheck("Compliance")
compliance_check = ComplianceCheck("Compliance_check")

iot_device_compliance = ComplianceCheck("IOT_device_compliance")
waiver_review_process = ComplianceCheck("Waiver_review_process")

# core_baseline = Process("CoreBaseline")
core_baseline = CoreBaseline("Core_Baseline")
cyber_security_baseline = CoreBaseline("Cyber_security_baseline")

cybersecurity_risk_management_approaches = cybersecurityRiskManagementApproaches("cybersecurity_risk_management_approaches")
implementation_of_guidelines = cybersecurityRiskManagementApproaches("implementation_of_guidelines")

disclosure_of_vulnerabilities = Process("Disclosure_of_vulnerabilities")
time_period = Process("Time_Period")
business_resources_for_development_and_support = Process("Customer_Support")
customer_support = Process("Customer_support")
third_party_services = Process("Third_party_services")

system_optimization_and_analytics = Process("System_optimization_and_analytics")
operational_assistance_to_agencies = Process("operational_assistance_to_agencies")
 
third_party_cloud_providers = Process("Third_party_cloud_providers")
device_optimization = Process("Device_optimization")

IoT_security_vulnerabilities = Process("IoT_Security_Vulnerabilities")
IoT_vulnerability = Process("IoT_Vulnerability")

cyber_security_vulnerabilities = Process("Cyber_Security_Vulnerabilities")
security_vulnerabilities = Process("security_vulnerabilities")
national_security = Process("National_security")
network_security_measures = Process("Network_security_measures")

rigorous_testing_protocols = Process("rigorous_testing_protocols")
expected_use_cases = Process("expected_use_cases")

mitigation_of_vulnerabilites = Process("mitigation_of_vulnerabilites")
vulnerability_detection_before_deployment = Process("vulnerability_detection_before_deployment")
firmware_update = Process("best_practices_for_firmware_updates")

what_to_communicate_to_customer = Process("what_to_communicate_to_customer")
customer_cybersecurity_needs_and_goals = Process("customer_cybersecurity_needs_and_goals")

how_to_address_customer_needs_and_goals = Process("customer_cybersecurity_needs_and_goals")
customers_risks = Process("customers_risks")
approaches_for_communicating_to_customers = Process("approaches_for_communicating_to_customers")

communicate_cybersecurity_risks = Process("cybersecurity_risk_management_approaches")
content_and_method_of_communication = Process("cybersecurity_risk_management_approaches")

updates = Updates("Updates-1")
application_of_updates = Updates("application_of_updates ")
updates_notifiation = Updates("updates_notifiation")
software_updates = Updates("software_updates")

manufacturer.data_property = [
    "Activity 1: Identify Customers and Users", 
    "Activity 2: Research Cybersecurity Needs",
    "Activity 3: Address Customer Needs",
    "Activity 4: Plan Support for Customer Needs",
    "Activity 5: Define Communication Approaches",
    "Activity 6: Decide Communication Content"
]

manufacturer.data_description = [
    "Define expected customers and use cases early in the design.",
    "Research customer cybersecurity needs and goals.",
    "Determine how to address customer needs and goals.",    
    "Plan for adequate support of customer needs and goals.",
    "Define approaches for communicating to customers.",
    "Decide what to communicate to customers and how."
]

# NIst8259 
# Add relationships between Stakeholders and Processes
manufacturer.performs.append(configuration_management)
manufacturer.shouldImplement.append(secure_software_architecture)
manufacturer.shouldImplement.append(rigorous_testing_protocols)
manufacturer.shouldImplement.append(network_security_measures)
manufacturer.shouldEstablish.append(secure_development_processes)
manufacturer.identify.append(customer)
manufacturer.define.append(expected_use_cases) 
manufacturer.determines.append(how_to_address_customer_needs_and_goals)
manufacturer.researches.append(customer_cybersecurity_needs_and_goals) 
manufacturer.understands.append(customers_risks)
manufacturer.provides.append(cybersecurity_capability) 
manufacturer.plansFor.append(customer_support)
manufacturer.provisions.append(hardware_and_software_resources)
manufacturer.considers.append(business_resources_for_development_and_support)
manufacturer.define.append(approaches_for_communicating_to_customers)
manufacturer.developsStrategies.append(communicate_cybersecurity_risks)
manufacturer.decides.append(what_to_communicate_to_customer)
manufacturer.determinesContent.append(content_and_method_of_communication)

NIST.isResponsibleFor.append(compliance_check)
NIST.developsAndUpdates.append(NIST_IR_8259)
NIST_IR_8259.definesAndUpdates.append(core_baseline)

core_baseline.encompasses.append(cybersecurity_capability)
core_baseline.derivedFrom.append(cybersecurity_risk_management_approaches)

secure_software_architecture.contain.append(mitigation_of_vulnerabilites)

rigorous_testing_protocols.include.append(vulnerability_detection_before_deployment)
secure_development_processes.include.append(firmware_update)
network_security_measures.include.append(firewalls)
network_security_measures.include.append(IDSs)
content_and_method_of_communication.include.append(cybersecurity_risk_management_approaches)

expected_use_cases.appliesTo.append(iot_device)

cybersecurity_capability.appliesTo.append(Means)
cybersecurity_capability.performedBy.append(iot_device)

CoreBaseline.canHelpManufacturersIdentify.append(CybersecurityCapability)
CybersecurityCapability.willBeDecidedAndShapedBy.append(Customer)

IoTDevice.willNeed.append(CybersecurityCapability)

CoreBaseline.doesNotInclude.append(NonCybersecurityCapability)

# Plaw integration
# Manufacturers
manufacturer.develop.append(iot_device)
manufacturer.adhereTo.append(cybersecurity_standards)
manufacturer.mustDesignFor.append(secure_development_processes)
manufacturer.mustEnsure.append(patch_availability)
manufacturer.mustCollaborateWith.append(government_agencies)

# IoT Devices
iot_device.mustMeet.append(minimum_security_standards)
iot_device.mustComplyWith.append(NIST_guidelines)
iot_device.areDefinedBy.append(NIST_draft_report)
iot_device.have.append(transducer)
iot_device.have.append(network_interface)
iot_device.differFrom.append(conventional_IT_devices)

# Federal Agencies
government_agencies.ownOrControl.append(iot_device)
government_agencies.use.append(iot_device)
government_agencies.mustComplyWith.append(NIST_guidelines)
government_agencies.cannotProcure.append(non_compliant_IoT_devices)
government_agencies.mustDisclose.append(cyber_security_vulnerabilities)
government_agencies.mustCoordinate.append(disclosure_of_vulnerabilities)
government_agencies.mustFollow.append(OMB_policies) 
government_agencies.collaborateWith.append(industry_experts) 
government_agencies.collaborateWith.append(educational_institutions)
government_agencies.mustImplement.append(configuration_management)

# NIST 
NIST.isTaskedToDevelop.append(IoT_security_standards_and_guidelines) 
NIST.publishes.append(security_vulnerabilities_reporting_guidelines)  
NIST_guidelines.mustAlignWith.append(ISO_standards_29147_and_30111)
NIST.mustReview.append(NIST_guidelines) # every 5 years data ppty for standard class
NIST.mustConsider.append(best_practices)

# Director of OMB 
dir_OMB.reviews.append(agency_security_policies)
dir_OMB.ensuresAlignmentWith.append(IoT_security_standards_and_guidelines)
dir_OMB.oversees.append(implementation_of_guidelines) 
dir_OMB.issues.append(policies_and_principles)
policies_and_principles.ensureConsistencyWith.append(IoT_security_standards_and_guidelines)
OMB_policies.doNotApplyTo.append(national_security_systems)
dir_OMB.establishes.append(waiver_review_process)

# Director of the Institute 
dir_NIST.develops.append(IoT_security_standards_and_guidelines)
dir_NIST.considers.append(best_practices)

# National Institute of Standards and Technology
NIST.develops.append(IoT_security_standards_and_guidelines)

# Secretary of Homeland Security 
secretary_HS.provides.append(operational_assistance_to_agencies)
secretary_HS.consultsWith.append(dir_OMB)

# Contractors 
contractors.mustReport.append(security_vulnerabilities)
contractors.mustComplyWith.append(NIST_guidelines)
contractors.mustAlignWith.append(Federal_acquisition_regulation)

# GAO and Committees
GAO.assesses.append(IoT_security_vulnerabilities)
GAO_report.isSubmittedTo.append(congressional_committees)
congressional_committees.receiveReportsFrom.append(compt_USA)
congressional_committees.address.append(IoT_efforts)

# Federal Acquisition Regulation
Federal_acquisition_regulation.mustAddress.append(security_vulnerabilities)

# Comptroller General
compt_USA.submitsReportsTo.append(congressional_committees)

# Miscellaneous 
dir_NIST.consultsWith.append(cybersecurity_researchers)
dir_NIST.consultsWith.append(industry_experts)
waivers.canBeGrantedFor.append(national_security_research)
waivers.canBeGrantedFor.append(educational_institutions)
waivers.canBeGrantedFor.append(cybersecurity_researchers)
CIO.review.append(iot_device_compliance)
congress.passes.append(IoT_cybersecurity_improvement_act)
IoT_cybersecurity_improvement_act.establishes.append(minimum_security_standards)

# Save ontology
onto.save(file="n8259a.owl", format="rdfxml")

In [ ]:
# Load manufacturer privacy policies
csv_path = r'C:\Users\policies_data.csv'
df_policies = pd.read_csv(csv_path)

# Retrieve Manufacturer class
Manufacturer = onto.search_one(iri="*Manufacturer")

if Manufacturer:
    print("Manufacturer class found! Creating instances...\n")
else:
    raise ValueError("Manufacturer class not found in ontology!")

# regex pattern for extracting company names
pattern = re.compile(r"(?:www\.)?([^.\/]+)\.(com|net|org|io|fi|ca|co|gov|edu|tech|us|store|pl|info|pw|eu|my|biz|cn)")

# Initialize manufacturer instances
manufacturer_instances = []

text_splitter_m = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

with onto:
    for index, row in df_policies.iterrows():

        file_name = str(row['file_name'])
        
        match = pattern.search(file_name)
        company_name = match.group(1) if match else file_name  
        company_name = re.sub(r'[\.\-]', '_', company_name)

        # vectorstore embedding (used llm embedding)
        embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
       
        # Manufacturer instances
        if Manufacturer:
            try:
                manufacturer_instance = Manufacturer(company_name)
                manufacturer_instance.policy_description = [row['policy_text']] if pd.notna(row['policy_text']) else ["No policy available"]
                
                company_name_policy = f"{company_name}: {manufacturer_instance.policy_description[0]}"
                
                text_chunks = text_splitter_m.create_documents([company_name_policy])                               
                vectorstore_m = DocArrayInMemorySearch.from_documents(text_chunks, embedding=embeddings)    
                
                manufacturer_instances.append(manufacturer_instance)
                manufacturer_count += 1               

            except Exception as e:
                print(f"Error creating {company_name}: {e}")

# Save ontology
onto.save(file="n8259a.owl", format="rdfxml")

In [ ]:
# Prompt for Entity-Relationship Extraction
entity_relationships_prompt = ChatPromptTemplate.from_template(
"""
You are an expert Entity-Relationship Extraction system.

Objective: Extract key relationships in the format:
Subject: [Entity A]  
Relation: [Predicate]  
Object: [Entity B]  
Description: [Brief explanation of the relationship]  

Context: {context}

Format Example:
- Subject: User Data  
- Relation: Collected by  
- Object: Manufacturer  
Description: The manufacturer collects personal data from users for security purposes.

- Subject: Secure Development  
- Relation: Ensures  
- Object: Cybersecurity Standards  
Description: Secure development processes help meet cybersecurity guidelines.

Now, extract multiple and comprehensive entity relationships from each of the provided manufacturer privacy policies.

"""
)

# Process manufacturers
for manufacturer in manufacturer_instances:
    if not manufacturer.policy_description or not manufacturer.policy_description[0]: 
        print(f"\nSkipping {manufacturer.name}, no policy description.")
        continue

    policy_text = manufacturer.policy_description[0]
    policy_chunks = text_splitter_m.create_documents([policy_text])      
    extracted_relationships = []

    for chunk in policy_chunks:
        chunk_text = chunk.page_content  

        # Call LLM 
        response = entity_relationships_prompt | model 
        extracted_chunk_relationships = response.invoke({"context": chunk_text})
        extracted_relationships.append(extracted_chunk_relationships)

    final_relationships = "\n".join(str(item.content) for item in extracted_relationships)

    # Append triples to data ppty
    if not hasattr(manufacturer, 'data_property'):
        manufacturer.data_property = []
    manufacturer.data_property.append(f"\nExtracted Entity-Relationships\n{final_relationships}")

# Save ontology 
onto.save(file="n8259a.owl", format="rdfxml")

In [ ]:
# Iterate through manufacturer instances
for manufacturer in manufacturer_instances:
        
    if not hasattr(manufacturer, 'data_property'):
        manufacturer.data_property = []
    if not hasattr(manufacturer, 'data_description'):
        manufacturer.data_description = []

    updated_data_property = []
    data_descriptions = []

    # Iterate through data properties for each manufacturer
    for entry in manufacturer.data_property:
        entry_content = str(entry)  

        # Extract triples and descriptions
        triples = []
        descriptions = []

        # Split by single newline to process each line
        lines = entry_content.split("\n") 
        current_triple = []
        for line in lines:
            line = line.strip()

            # Check if it's part of a triple
            if line.startswith("- Subject:") or line.startswith("- Relation:") or line.startswith("- Object:"):
                current_triple.append(line) 
            
            if len(current_triple) == 3:
                triples.append(" ".join(current_triple))  
                current_triple = []  

            # Check if it's a description after a triple
            elif line.lower().startswith("description:"):
                descriptions.append(line) 
        
        updated_data_property.extend(triples)        
        data_descriptions.extend(descriptions)

    manufacturer.data_property = updated_data_property
    manufacturer.data_description = data_descriptions

In [ ]:
for manufacturer in manufacturer_instances:
    print("\nManufacturer:", manufacturer.name)
    if hasattr(manufacturer, 'data_property') and manufacturer.data_property:
        print("Data:")
        print("Data Property:", manufacturer.data_property) 
        print("Data Description:", manufacturer.data_description)  
        print("Original Policy Text:", manufacturer.policy_description)

In [ ]:
# object and data properties in Manufacturer class
object_properties = []
for prop in onto.object_properties():
    
    if Manufacturer in prop.domain or Manufacturer in prop.range:
        object_properties.append(prop)

data_properties = []
for prop in onto.data_properties():
    
    if Manufacturer in prop.domain:
        data_properties.append(prop)

print("Object Properties in Manufacturer class:")
for prop in object_properties:
    print(f"- {prop.name}")

print("\nData Properties in Manufacturer class:")
for prop in data_properties:
    print(f"- {prop.name}")

In [ ]:
output_string = ""

# Extract object properties in Manufacturer class
object_properties = []
for prop in onto.object_properties():
    if Manufacturer in prop.domain or Manufacturer in prop.range:
        object_properties.append(prop)

# Object properties in Manufacturer class with full relationships
output_string += "Object Properties in Manufacturer class with full relationships:\n"

for prop in object_properties:
    
    domain_classes = set(cls.name for cls in prop.domain)
    range_classes = set(cls.name for cls in prop.range)

    # Append to output string
    output_string += f"\n- Object Property: {prop.name}\n"
    output_string += f"  Domain: {', '.join(domain_classes)}\n"
    output_string += f"  Range: {', '.join(range_classes)}\n"

    related_instances = set()

    # Iterate through all instances
    for manufacturer in onto.Manufacturer.instances():
        
        linked_instances = getattr(manufacturer, prop.name, [])

        if not isinstance(linked_instances, list):
            linked_instances = [linked_instances]

        for instance in linked_instances:
            if instance in onto.Process.instances(): 
                related_instances.add(instance.name)

    # linked instances
    if related_instances:
        output_string += "  Linked Instances:\n"
        for instance_name in related_instances:
            output_string += f"  Instance: {instance_name}\n"

In [ ]:

# regex to get object properties
pattern = r"- Object Property: (\S+)\s+Domain: ([^\n]+)\s+Range: ([^\n]+)(?:\s+Linked Instances:\s*((?:\s*Instance: .+\n?)*))?"

object_properties = []

matches = re.findall(pattern, output_string)

for object_property, domain_str, range_str, instances_block in matches:
    domains = [d.strip() for d in domain_str.split(",")]
    ranges = [r.strip() for r in range_str.split(",")]
    instances = re.findall(r"Instance: (\S+)", instances_block or "")
    
    for domain in domains:
        for range_ in ranges:
            if instances:
                for inst in instances:
                    object_properties.append((domain, object_property, inst))
            else:
                # use range
                object_properties.append((domain, object_property, range_))

print(object_properties)

In [ ]:
for subj, pred, obj in object_properties:
    
    try:
        domain_class = getattr(onto, subj)
    except AttributeError:
        print(f"Class {subj} not found in ontology — skipping")
        continue
    
    if not hasattr(domain_class, "object_property"):
        domain_class.object_property = []

    triple = f"{subj} {pred} {obj}"
    
    if triple not in domain_class.object_property:
        domain_class.object_property.append(triple)

onto.save("n8259a.owl")

In [ ]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Prompt Template 
description_comparison_prompt = ChatPromptTemplate.from_template(
    """
    Compare the following triple activities and determine how similar they are:

    Description 1: {description_1}

    Description 2: {description_2}

    Provide:
    1. A numeric similarity score from 0 to 100.
    2. An explanation of the similarity/differences.

    Format:
    Score: <numeric_value>
    Explanation: <your_explanation>
    """
)

# Cosine Similarity function
def compute_similarity(text1, text2):
    if not text1 or not text2:
        return 0.0
    embedding_1 = embedding_model.encode([text1])
    embedding_2 = embedding_model.encode([text2])
    return cosine_similarity(embedding_1, embedding_2)[0][0]

def compare_descriptions(description_1, description_2):
    if not description_1 or not description_2:
        return 0.0, "Insufficient text for comparison."

    response = (description_comparison_prompt | model).invoke({
        "description_1": description_1,
        "description_2": description_2
    })

    explanation_text = response.content.strip()
    lines = explanation_text.split("\n")
    score_line = next((line for line in lines if line.lower().startswith("score:")), None)

    try:
        similarity_score = float(score_line.replace("Score:", "").strip()) if score_line else 0.0
    except Exception:
        similarity_score = 0.0
        explanation_text += "\n(Note: Failed to parse score.)"

    return similarity_score, explanation_text

#  Comparison Loop
def compare_all_manufacturers(reference_items, manufacturers, threshold=0.47):
    for manufacturer in manufacturers:
        name = manufacturer.name
        manufacturer_items = manufacturer.data_property or []

        similarity_scores = []
        matched_properties = []
        unmatched_properties = []

        for ref_item in reference_items:
            best_similarity = 0.0
            best_match = None

            for item in manufacturer_items:
                sim = compute_similarity(ref_item, item)
                if sim > best_similarity:
                    best_similarity = sim
                    best_match = item

            similarity_scores.append(best_similarity)

            if best_similarity >= threshold:
                matched_properties.append((ref_item, best_match, best_similarity))
            else:
                unmatched_properties.append((ref_item, best_match, best_similarity))

        avg_similarity = sum(similarity_scores) / len(similarity_scores) if similarity_scores else 0.0
        final_status = "PASS" if avg_similarity >= threshold else "FAIL"

        # Ensure fields exist
        if not hasattr(manufacturer, 'status'):
            manufacturer.status = []
        if not hasattr(manufacturer, 'compliance_report'):
            manufacturer.compliance_report = []

        manufacturer.status.append(final_status)
        manufacturer.compliance_report.append(float(avg_similarity))

        print(f"{name} Avg Similarity: {avg_similarity*100:.2f}% | Status: {final_status}")

# Comparison function
reference_data = manufacturer_1.data_property or []
reference_objects = manufacturer_1.object_property or []
reference_items = reference_data + reference_objects

compare_all_manufacturers(reference_items, manufacturers)

In [ ]:
with onto:
    results = list(default_world.sparql("""
    PREFIX : <http://example.org/onto.owl#>
    SELECT DISTINCT ?property
    WHERE {
        ?property rdfs:domain :Manufacturer .
    }
    """))

for r in results:
    print(f"Property: {r[0].name}")

In [ ]:
with onto:
    results = list(default_world.sparql("""
    PREFIX : <http://example.org/onto.owl#>
    SELECT DISTINCT ?manufacturer
    WHERE {
        ?manufacturer a :Manufacturer ;
                      :status "FAIL" .
    }
    """))

for r in results:
    manufacturer_uri = r[0].name 

    matching = [m for m in manufacturers if m.name == manufacturer_uri]
    
    if matching:
        score = matching[0].compliance_report[0]
        print(f"Non-compliant Manufacturer: {manufacturer_uri} | Score: {score:.2f}")
    else:
        print(f"Non-compliant Manufacturer: {manufacturer_uri} | Score: Not found")


In [ ]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

y_true = [1 if "PASS" in m.status[0] else 0 for m in manufacturers]
y_scores = [m.compliance_report[0] for m in manufacturers]

roc_auc = roc_auc_score(y_true, y_scores)
print(f"ROC-AUC Score: {roc_auc:.3f}")

# Threshold tuning
thresholds = [round(x * 0.01, 2) for x in range(30, 81)]
best_threshold = 0.0
best_f1 = 0.0

threshold_metrics = []

for threshold in thresholds:
    y_pred = [1 if score >= threshold else 0 for score in y_scores]
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    
    threshold_metrics.append((threshold, precision, recall, f1))
    
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f"\nBest Threshold: {best_threshold:.2f}")

In [ ]:
import matplotlib.pyplot as plt

metrics_df = pd.DataFrame(threshold_metrics, columns=['Threshold', 'Precision', 'Recall', 'F1'])

plt.figure(figsize=(10, 6))
plt.plot(metrics_df['Threshold'], metrics_df['Precision'], label='Precision', marker='o')
plt.plot(metrics_df['Threshold'], metrics_df['Recall'], label='Recall', marker='s')
plt.plot(metrics_df['Threshold'], metrics_df['F1'], label='F1 Score', marker='^')

plt.axvline(x=best_threshold, color='gray', linestyle='--', label=f'Best Threshold ({best_threshold:.2f})')

plt.title("Evaluation Metrics vs Threshold")
plt.xlabel("Threshold")
plt.ylabel("Score")
plt.ylim(0, 1.05)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
